In [ ]:
if s.FILE_SIZE in ("Medium", "Large"):
    NUM_PROCS = 10

In [39]:
window_df.loc[:, "window_delta"] = (
    window_df["timestamp"] - window_df["timestamp"].min()
).dt.total_seconds() + 1

In [40]:
# Because of data quality issue

in_scope_window_size = (
    min([window_df["timestamp"].max(), complete_data_present_till]) -
    window_df["timestamp"].min()
).total_seconds()
if in_scope_window_size < 1:
    in_scope_window_size = 1

85500

In [41]:
in_scope_edges = pov_df.groupby(["source", "target"]).agg(is_laundering=("is_laundering", "max")).reset_index()
in_scope_sources = set(in_scope_edges["source"])
in_scope_targets = set(in_scope_edges["target"])
in_scope_nodes = list(in_scope_sources.union(in_scope_targets))

In [42]:
def aggregate_edges(data_input):
    data_aggregated = (
        data_input.groupby(["source", "target"])
        .agg(
            amount=("amount", "sum")
        )
    ).reset_index()

    source_totals = (
        data_aggregated.groupby("source")
        .agg({"amount": "sum"})["amount"]
        .to_dict()
    )
    target_totals = (
        data_aggregated.groupby("target")
        .agg({"amount": "sum"})["amount"]
        .to_dict()
    )

    data_aggregated.loc[:, "total_sent_by_source"] = data_aggregated.loc[
        :, "source"
    ].apply(lambda x: source_totals[x])
    data_aggregated.loc[:, "total_received_by_target"] = data_aggregated.loc[
        :, "target"
    ].apply(lambda x: target_totals[x])
    data_aggregated.loc[:, "weight"] = data_aggregated.apply(
        lambda x: (
            (x["amount"] / x["total_sent_by_source"])
            + (x["amount"] / x["total_received_by_target"])
        ),
        axis=1,
    )
    return data_aggregated.loc[:, ["source", "target", "weight"]]

In [43]:
window_edges = aggregate_edges(window_df)

In [44]:
graph = ig.Graph.DataFrame(window_edges, use_vids=False, directed=True)

In [45]:
communities = get_communities_multi_proc(in_scope_nodes, graph, NUM_PROCS)
sizes = [len(x[1]) for x in communities]
round(np.mean(sizes)), round(np.max(sizes)), sum(sizes)

CPU times: user 414 ms, sys: 96.1 ms, total: 510 ms
Wall time: 15.6 s


(11, 75, 755713)

In [ ]:
if s.FILE_SIZE in ("Medium", "Large"):
    NUM_PROCS = 6

In [ ]:
columns = list(window_df.columns)
columns.remove("transaction_id")
columns.remove("target_currency")
columns.remove("target_amount")
columns.remove("source")
columns.remove("target")
columns = ["source", "target"] + columns

graph = ig.Graph.DataFrame(window_df[columns], use_vids=False, directed=True)

In [ ]:
iterator_chunk_as_pickles, param_locations = create_workload_for_multi_proc(
    len(communities),
    communities, 
    NUM_PROCS, 
    graph,
    shuffle=True
)
features = get_features_multi_proc(
    iterator_chunk_as_pickles, param_locations[0], "features.get_features_chunk_with_gf",
    reset_staging=False
)
features.loc[:, "in_scope_window_size"] = in_scope_window_size
features.columns = [f"{s.G_COMM_PREFIX}{x}" if x != "key" else x for x in features.columns]

In [51]:
in_scope_trx_communities = window_df.set_index("source").join(
    pd.DataFrame(index=list(in_scope_sources)), how="inner"
).reset_index()

In [52]:
iterator_chunk_as_pickles, _ = create_workload_for_multi_proc(
    in_scope_trx_communities["source"].nunique(),
    in_scope_trx_communities.groupby("source"), 
    NUM_PROCS, shuffle=False
)
features_source = get_features_multi_proc(
    iterator_chunk_as_pickles, None, "features.get_features_chunk_without_gf",
    reset_staging=False
)
features_source.columns = [f"{s.G_1HOP_PREFIX}{x}" if x != "key" else x for x in features_source.columns]

CPU times: user 2.86 s, sys: 221 ms, total: 3.08 s
Wall time: 38.3 s


In [ ]:
iterator_chunk_as_pickles, _ = create_workload_for_multi_proc(
    in_scope_trx_communities["target"].nunique(),
    in_scope_trx_communities.groupby("target"), 
    NUM_PROCS, shuffle=False
)
features_target = get_features_multi_proc(
    iterator_chunk_as_pickles, None, "features.get_features_chunk_without_gf",
    reset_staging=False
)
features_target.columns = [f"{s.G_1HOP_PREFIX}{x}" if x != "key" else x for x in features_target.columns]

In [ ]:
def reset_index(df_input, index_name):
    df_input.index.name = index_name
    df_input = df_input.reset_index()
    return df_input

In [ ]:
all_features = features.set_index("key").join(
    features_source.set_index("key"), how="left", rsuffix=f"_1_hop_as_source"
)
all_features = reset_index(all_features, "key") 

all_features = all_features.set_index("key").join(
    features_target.set_index("key"), how="left", rsuffix=f"_1_hop_as_target"
)